In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *
from pyspark.sql.functions import col, date_format, year, quarter, month, weekofyear

StatementMeta(, 01e6742f-0eff-4e0f-a654-579bfcee39c8, 3, Finished, Available, Finished)

###### **<mark>Creating dimension table of Shipmode</mark>**

In [2]:
%%sql
create table if not exists Ecommerce_project.Gold_Shipmode
(
    Ship_Mode string,
    ShipMode_ID int,
    Created_TS TIMESTAMP,
    Modified_TS TIMESTAMP
)
USING DELTA 

StatementMeta(, 01e6742f-0eff-4e0f-a654-579bfcee39c8, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

###### **<mark>Build incremental load</mark>**

In [3]:
Max_Date=spark.sql("select coalesce(max(Modified_TS),'1900-01-01') from Ecommerce_project.gold_shipmode").first()[0]

StatementMeta(, 01e6742f-0eff-4e0f-a654-579bfcee39c8, 5, Finished, Available, Finished)

###### **<mark>Creating dataframe with required columns for gold Ship_Mode table</mark>**

In [4]:
df_bronze=spark.sql("select distinct Ship_Mode from Ecommerce_project.bronze_sales where Modified_TS>'{}'".format(Max_Date))

StatementMeta(, 01e6742f-0eff-4e0f-a654-579bfcee39c8, 6, Finished, Available, Finished)

###### **<mark>Creating logic for ShipMode_ID column</mark>**

In [5]:
Max_ID = spark.sql("select coalesce(max(ShipMode_ID),0) from Ecommerce_project.gold_shipmode").first()[0]

StatementMeta(, 01e6742f-0eff-4e0f-a654-579bfcee39c8, 7, Finished, Available, Finished)

###### <mark>**Adding ShipMode_ID to dataframe using monotonically_increasing_id**</mark>

In [6]:
final_df= df_bronze.withColumn('ShipMode_ID',monotonically_increasing_id()+Max_ID+1)

StatementMeta(, 01e6742f-0eff-4e0f-a654-579bfcee39c8, 8, Finished, Available, Finished)

###### **<mark>Creating view from final_df</mark>**

In [7]:
final_df.createOrReplaceTempView('ViewShipMode')

StatementMeta(, 01e6742f-0eff-4e0f-a654-579bfcee39c8, 9, Finished, Available, Finished)

###### <mark>**Inserting data from view to Gold_ShipMode Table using Merge Query**</mark>

In [8]:
%%sql
MERGE INTO Ecommerce_project.gold_shipmode as gs
USING ViewShipMode as vs 
on gs.Ship_Mode = vs.Ship_Mode

WHEN MATCHED THEN
UPDATE SET
Modified_TS = CURRENT_TIMESTAMP()

WHEN NOT MATCHED THEN
INSERT
(
gs.Ship_Mode,
gs.ShipMode_ID,
gs.Created_TS,
gs.Modified_TS
)
VALUES
(
vs.Ship_Mode,
vs.ShipMode_ID,
CURRENT_TIMESTAMP(),
CURRENT_TIMESTAMP
)

StatementMeta(, 01e6742f-0eff-4e0f-a654-579bfcee39c8, 10, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>